# The Cognitive Atlas & Open-Access Neuroimaging Data

![http://cognitiveatlas.org//static/img/logo-front.png](http://cognitiveatlas.org//static/img/logo-front.png)


<hr>

The Cognitive Atlas web interface is useful, but limited in what we are able to analyze, in a large scale sense. Here we are going to interact with the [Cognitive Atlas](https://www.cognitiveatlas.org) 
via its [Python API](https://github.com/CognitiveAtlas/cogat-python).

We'll use this database, along with Neurosynth, to address the following questions:

 1. What is working memory?
 2. What concepts are related to it?
 3. What parts of the brain have associations from the literature?

<hr>


**API means "application programming interface"** and it really is exactly that: a set of functions that you can call from a programming language (e.g., Python and R are common languages for scientific programming) to interact with a resource (for example, a database) to answer some question.

In fact, APIs are essential for science - there are questions that, in order to answer, might require interaction with datasets beyond the ability of a human. We can't easily answer all queries that we might have about cognitive knowledge in a web browser.

For example, here are some more advanced use cases that would require programming and (some) graph theory:

 - What cognitive concepts are represented across a database of over 1 million articles from the literature
 - **how** similar is working memory to all other concepts in the atlas?
     - Can you give me a number?
     - Can you define a threshold of significance?
 - What parts of the brain, represened by coordinate locations reported in the literature, are associated with the concept?

Beyond using the Cognitive Atlas as a lookup for terms, you can see that there are many use cases in research that would warrant interacting with it programatically, integrated with other tools, or scaled. Today let's explore one of these use cases - the third - and understand what areas of the brain might be involved with working memory.

## Step 1. Prepare the API.
First, you'll need to open a terminal window within the DataHub and install the Cognitive Atlas tools with the following line. You might need to restart this kernel after doing so.

```bash
pip install --user cognitiveatlas
```

**NB:** the API itself is a [RESTful](https://codewords.recurse.com/issues/five/what-restful-actually-means) API, and could be interacted with via any programming language that can parse the [JSON endpoints](https://cognitiveatlas.org/api).

## Step 2. Query the Cognitive Atlas API

For those questions above, we're going to take the following approach:

 1. Use the Cognitive Atlas Python API to query the Cognitive Atlas to look up a term based on it's name
 2. From 1., use the term identifier, and use that identifier to look for terms that are related.
 3. Use the same identifier to query the NeuroSynth API to derive a brain map.
 
Let's import functions that we need from the Cognitive Atlas API, namely those to "get" things.

In [1]:
from cognitiveatlas.api import get_concept, get_task, get_disorder

How would you know how to use these functions? if you follow the link from the Cognitive Atlas site
you are taken to the [readthedocs](https://cogat-python.readthedocs.io/en/latest/getting_started.html#get-concept) pages. ReadTheDocs is a service that can help software maintainers to easily render and provide beautiful
documentation for their users. This is where we will find examples, and how to provide arguments to a function.

If a piece of software _doesn't_ provide this, you should ask for it. Whether that is a post on an
issue board or an emali to a list, as a user it's an important resource to be provided for you.
And it's ok to ask for what you need. Based on the page linked above for the function `get_concept`
let's see if we get get the concept "working memory":

In [2]:
concept = get_concept(name="working memory")

http://cognitiveatlas.org/api/v-alpha/concept?name=working%20memory
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


The call is showing us a quick link to see the concept, along with how to interact with it. It looks like the result has a [pandas](https://pandas.pydata.org/) data frame, a [JSON](http://json-schema.org/) (JavaScript Object Notation) object, and the url as a string. Let's see what we get when we print it.

Note that if you didn't know what was in the result (and the documentation didn't help you) you could type `dir(concept)` to see it, or in an interactive jupyter notebook or ipython terminal you could use TAB autocompletion.

In [3]:
# Print the keys for the dictionary
print("The keys for the concept %s are:" % concept.json['name'])
print(concept.json.keys())

The keys for the concept working memory are:
dict_keys(['id_user', 'contrasts', 'citations', 'definition_text', 'id_concept_class', 'name', 'event_stamp', 'relationships', 'conceptclasses', 'alias', 'type', 'creation_time', 'def_id', 'last_updated', 'concepts', 'def_event_stamp', 'def_id_user', 'id'])


And then we can look at the entire thing, if we wish, which includes the values for those keys!

In [4]:
concept.json

{'id_user': 'usr_0000000000',
 'contrasts': [{'id_user': 'usr_4cf5de07654c8',
   'creation_time': 1512660661740,
   'relationship': 'MEASUREDBY',
   'last_updated': 1512660661740,
   'name': 'dual task classification learning trials',
   'event_stamp': '2011-11-11T04:53:26',
   'id': 'cnt_4ebcaa463d20e'},
  {'id_user': 'usr_51bc76ae9636c',
   'creation_time': 1512660683430,
   'relationship': 'MEASUREDBY',
   'last_updated': 1512660683430,
   'name': 'Cue',
   'event_stamp': '2015-04-17T18:34:14',
   'id': 'cnt_55315226076d3'},
  {'id_user': 'usr_4de5345b3b6a8',
   'creation_time': 1512660668612,
   'relationship': 'MEASUREDBY',
   'last_updated': 1512660668612,
   'name': 'number correct',
   'event_stamp': '2013-03-28T18:56:13',
   'id': 'cnt_5154924daead4'},
  {'id_user': 'usr_4de5345b3b6a8',
   'creation_time': 1512660668579,
   'relationship': 'MEASUREDBY',
   'last_updated': 1512660668579,
   'name': 'false positives',
   'event_stamp': '2013-03-28T18:49:54',
   'id': 'cnt_515490

That's quite a bit of metadata and information about "working memory" that we just didn't see in the web interface. This also tends to be the case for APIs - they provide a richer source of information. We can easily see the definition of working memory under the "definition" key:

In [5]:
print(concept.json['definition_text'])

active maintenance and flexible updating of goal/task relevant information (items, goals, strategies, etc.) in a form that resists interference but has limited capacity. These representations may involve flexible binding of representations, may be characterized by the absence of external support for the internally maintained representations, and are frequently temporary due to ongoing interference



In [6]:
my_concept = get_concept(name="emotion")
print(my_concept.json['definition_text'])

http://cognitiveatlas.org/api/v-alpha/concept?name=emotion
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>
a complex of psychological phenomena that involve some degree of arousal and valence (positive/negative)


Boom, same defintion. What if you had a list of terms that you would otherwise have to manually look up?

<div class="alert alert-success"><b>Task</b>: Write a <code>for loop</code> to go through three different terms of your choosing and print the defintions.</div>

It's possible you found a term that isn't defined yet! If I were interested or working on this term, I would want to open up discussion on the website with others to define this term. We can also use the API to look up concepts based on other fields, or the same lookup for tasks or disorders:

In [7]:
task = get_task(name="Stroop task")

http://cognitiveatlas.org/api/v-alpha/task?name=Stroop%20task
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


In [8]:
phenotype = get_disorder(name="major depressive disorder")

http://cognitiveatlas.org/api/v-alpha/disorder?name=major%20depressive%20disorder
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


At this point we've successfully used the API to answer our first question: *what is working memory.* You also have a sense of how this could scale. Let's move on to the second question: *what concepts are related to working memory?*

<hr>

## Step 3. Look for related terms

Our next task is to look for related terms. Let's again get our concept, and switch it up a bit and look at "anxiety" instead. We will get the concept, and then look at the relationships to it. This would be akin to manually walking down a link on the graph to find another node.

In [9]:
concept = get_concept(name='anxiety')
concept.json['relationships']

http://cognitiveatlas.org/api/v-alpha/concept?name=anxiety
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


[{'id_user': 'usr_0000000000',
  'relationship': 'KINDOF',
  'definition_text': 'a complex of psychological phenomena that involve some degree of arousal and valence (positive/negative)',
  'id_concept_class': 'ctp_C8',
  'name': 'emotion',
  'event_stamp': '2009-06-22T19:12:55',
  'direction': 'parent',
  'alias': '',
  'creation_time': 1512660629809,
  'def_id': 'def_4f7350605133e',
  'last_updated': 1512660629809,
  'def_event_stamp': '2012-03-28T17:54:40',
  'def_id_user': 'usr_49a467bf4e0db',
  'id': 'trm_4a3fd79d0a17f'}]

Above we learn that anxiety is a `KIND OF` emotion. We could then look up this term (the walking along the graph part):

In [10]:
parent = get_concept(concept.json['relationships'][0]['id'])

http://cognitiveatlas.org/api/v-alpha/concept?id=trm_4a3fd79d0a17f
Result Includes:<pandas:data frame><json:dict><txt:str><url:str>


## Step 4. Derive a Brain Map

<div class="alert alert-success"><b>Task</b>:
    
Use the [NeuroSynth](http://neurosynth.org) web interface to look up a brain map based on a concept term of interest. For example, [here is the map](http://neurosynth.org/analyses/terms/anxiety/) derived using a summary meta analysis method (based on coordinates reported in the literature) for the term "anxiety". </div>

> You can think of NeuroSynth as a framework to do a meta analysis that looks across the psychology literature, finds tables of x,y,z coordinates reported in publications, and looks for "significant" areas of agreement. In other words, what blobs in the brain are commonly seen in papers with a high prevalence of the term "anxiety" in the abstract? It's a fairly simple method, but it works well.


## Step 5. Show some brains (just for fun)

Because I promised you'd learn how to code while looking at brains, let's use a package called `nilearn` and some of their sample data to see some brainsssss.

In [4]:
# Import the necessary packages and set up the plotting.
from nilearn import datasets, plotting, image, surface
%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'nilearn'

First, we can look at the MNI template, which is used to standardize brain images.

In [ ]:
# Let us use a Nifti file that is shipped with nilearn
from nilearn.datasets import MNI152_FILE_PATH

plotting.plot_img(MNI152_FILE_PATH)
plt.show()

By interacting with Neurovault, we can grab some task specific images from [this collection](https://neurovault.org/images/10426/)! The following contains a set of images for left versus right presses. Notice the differences in the left and right hemisphere.

If you want, you can also try `datasets.fetch_neurovault_auditory_computation_task()`.

In [ ]:
motor_images = datasets.fetch_neurovault_motor_task()
motor_images.images
tmap_filename = motor_images.images[0]
plotting.plot_stat_map(tmap_filename)
plt.show()

In [ ]:
# We can threshold these images
plotting.plot_stat_map(tmap_filename, threshold=3)
plt.show()

In [ ]:
# We can plot across various sagittal sections
plotting.plot_stat_map(tmap_filename, display_mode='x', threshold=1.,
                       cut_coords=range(0, 51, 10), title='Slices')
plt.show()

In [ ]:
# We can smooth the images
smoothed_img = image.smooth_img(tmap_filename, fwhm = 3)
plotting.plot_stat_map(smoothed_img)
plt.show()

In [ ]:
# We can create a 3D volume of the image
fsaverage = datasets.fetch_surf_fsaverage()
texture = surface.vol_to_surf(tmap_filename, fsaverage.pial_right)
plotting.plot_surf_stat_map(fsaverage.infl_right, texture, hemi='right',
                            title='Surface right hemisphere', colorbar=True,
                            threshold=1., bg_map=fsaverage.sulc_right)
plt.show()

Many more fun things on the [Nilearn usage examples page](https://nilearn.github.io/auto_examples/index.html)!

## About this notebook

This notebook is largely derived from [this lesson](https://github.com/vsoch/dinosaur-lessons/blob/master/neuro/notebook/cognitive-atlas-intro.ipynb), developed by [vsoch](https://github.com/vsoch). 